In [1]:
from autopycoin.data import random_ts
from autopycoin.models import PoolNBEATS, create_interpretable_nbeats
from autopycoin.dataset import WindowGenerator
import tensorflow as tf
import pandas as pd

data = random_ts(n_steps=1000,
                 trend_degree=2,
                 periods=[10],
                 fourier_orders=[10],
                 trend_mean=0,
                 trend_std=1,
                 seasonality_mean=0,
                 seasonality_std=1,
                 batch_size=1,
                 n_variables=1,
                 noise=True,
                 seed=42)
data = pd.DataFrame(data[0].numpy(), columns=['test'])

w = WindowGenerator(
       input_width=70,
       label_width=10,
       shift=10,
       test_size=50,
       valid_size=10,
       flat=True,
       batch_size=32,
       preprocessing=lambda x,y: (x, (x, y))
    )

w = w.from_array(data=data,
       input_columns=['test'],
       label_columns=['test'])

nbeats_model = lambda label_width: create_interpretable_nbeats(label_width=label_width, 
seasonality_n_neurons=20, trend_n_neurons=20)

model = PoolNBEATS(
            label_width=10,
            n_models=1,
            nbeats_models=nbeats_model,
            )
model.compile(tf.keras.optimizers.Adam(
   learning_rate=0.0015, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True,
   name='Adam'), loss=['mse', 'mae'], metrics=['mae'])
history = model.fit(w.train, validation_data=w.valid, epochs=1, verbose=1)

model.predict(w.test.take(1))

1 Physical GPUs, 1 Logical GPUs


In [ ]:
len(model.predict(w.test.take(1))[1])

TypeError: in user code:

    File "C:\Users\gaetd\anaconda3\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\gaetd\anaconda3\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\gaetd\anaconda3\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\gaetd\OneDrive\Bureau\github_perso\autopycoin\autopycoin\models\pool.py", line 248, in predict_step
        y = self.postprocessing_y(y)
    File "c:\Users\gaetd\OneDrive\Bureau\github_perso\autopycoin\autopycoin\models\nbeats.py", line 948, in postprocessing_y
        return self.fn_agg(y)

    TypeError: Failed to convert elements of [UnivariateTensor(values=<tf.Tensor 'pool_nbeats/interpretable_NBEATS/Add_1:0' shape=(None, 10) dtype=float32>, quantiles=Fa

In [ ]:
        # aggregation if possible
        if all(inputs_reconstructed[0].shape == tensor.shape for tensor in inputs_reconstructed[1:]):
            inputs_reconstructed = self._fn_agg(inputs_reconstructed, axis=-1)

        # aggregation if possible
        if all(outputs[0].shape == tensor.shape for tensor in outputs[1:]):
            outputs = self._fn_agg(outputs, axis=-1)